# 自定义层

深度学习成功背后的一个因素是神经网络的灵活性：
我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。
例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。
有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。
在这些情况下，必须构建自定义层。本节将展示如何构建自定义层。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。
回忆一下在[块和层](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/04.deep-learning-computation/02.model-construction.ipynb)对块的介绍，
这应该看起来很眼熟。
下面的`CenteredLayer`类要从其输入中减去均值。
要构建它，我们只需继承基础层类并实现前向传播功能。

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

让我们向该层提供一些数据，验证它是否能按预期工作。


In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以[**将层作为组件合并到更复杂的模型中**]。

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(4.6566e-09, grad_fn=<MeanBackward0>)

## [**带参数的层**]

以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层，
这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如管理访问、初始化、共享、保存和加载模型参数。
这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。
在此实现中，我们使用修正线性单元作为激活函数。
该层需要输入参数：`in_units`和`units`，分别表示输入数和输出数。

In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.rand(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化`MyLinear`类并访问其模型参数。

In [6]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.3033, -0.4409, -0.2735],
        [-0.7412,  0.1353, -0.2788],
        [-1.1309, -0.5983,  0.3643],
        [-1.3335, -0.2788,  1.0762],
        [-0.1278, -1.0522,  0.9761]], requires_grad=True)

我们可以[**使用自定义层直接执行前向传播计算**]

In [7]:
linear(torch.rand(2, 5))

tensor([[0.0140, 0.0883, 1.1155],
        [0.0000, 0.0000, 2.4038]])

我们还可以(**使用自定义层构建模型**)，就像使用内置的全连接层一样使用自定义层。

In [8]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[5.2781],
        [5.6565]])

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。

### 练习一

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。

&emsp;&emsp;这个公式表示一个线性变换，将输入张量$x$中所有可能的二元组$(x_i,x_j)$进行组合，并对它们进行加权求和。其中，$W_{ijk}$表示权重张量中第$i,j,k$个元素的值。具体而言，该公式计算了输入张量$x$中所有二元组$(x_i, x_j)$对应的特征向量$u_{ij}$：


$$
u_{ij} = x_i \cdot x_j
$$


&emsp;&emsp;然后，根据权重张量$W$中的权重$W_{ijk}$，对所有特征向量$u_{ij}$进行线性组合，得到输出向量$y_k$为：


$$
y_k = \sum_{i,j} W_{ijk} u_{ij} = \sum_{i,j} W_{ijk} x_i x_j
$$


&emsp;&emsp;该操作可以被视为一种降维操作，将高维输入$x$映射到低维输出空间$y$中。

In [9]:
import torch
import torch.nn as nn

class TensorReduction(nn.Module):
    def __init__(self, dim1, dim2):
        super(TensorReduction, self).__init__()
        # 定义一个可训练的权重参数，维度为(dim2, dim1, dim1)
        self.weight = nn.Parameter(torch.rand(dim2, dim1, dim1))

    def forward(self, X):
        # 初始化一个全零张量，大小为(X.shape[0], self.weight.shape[0])
        Y = torch.zeros(X.shape[0], self.weight.shape[0])
        for k in range(self.weight.shape[0]):
            # 计算temp = X @ weight[k] @ X^T
            temp = torch.matmul(X, self.weight[k]) @ X.T
            # 取temp的对角线元素，存入Y[:, k]
            Y[:, k] = temp.diagonal()
        return Y

# 创建一个TensorReduction层，dim1=10，dim2=5
layer = TensorReduction(10, 5)
# 创建一个大小为(2, 10)的张量X
X = torch.rand(2, 10)
# 对layer(X)进行前向传播，返回一个大小为(2, 5)的张量
print(f'X.shape:{X.shape}')
print(f'X:{X}')
print(f'layer(X).shape:{layer(X).shape}')
print(f'layer(X):{layer(X)}')

X.shape:torch.Size([2, 10])
X:tensor([[0.5466, 0.3143, 0.1361, 0.0980, 0.8118, 0.2218, 0.7450, 0.1392, 0.6839,
         0.7425],
        [0.9925, 0.1745, 0.3300, 0.8665, 0.8253, 0.4589, 0.4624, 0.0362, 0.7628,
         0.8761]])
layer(X).shape:torch.Size([2, 5])
layer(X):tensor([[ 9.5115, 10.5971,  9.0085, 10.7240,  9.9433],
        [16.9628, 16.8431, 15.5624, 17.8134, 17.1410]], grad_fn=<CopySlices>)


把(2, 10)的张量降维到(2, 5)

### 练习二

2. 设计一个返回输入数据的傅立叶系数前半部分的层。

&emsp;&emsp;傅里叶级数可以参考[维基百科](https://en.wikipedia.org/wiki/Fourier_series)中的定义。

&emsp;&emsp;在`torch`中有相应的函数可以轻松实现傅里叶级数，如下代码所示：

In [10]:
import torch.nn as nn
import torch.fft as fft

class FourierLayer(nn.Module):
    def __init__(self):
        super(FourierLayer, self).__init__()

    def forward(self, x):
        # 对输入的张量 x 进行快速傅里叶变换
        x = fft.fftn(x)
        # 取出第三个维度的前半部分，即去掉直流分量和镜像分量
        x = x[:, :, :x.shape[2] // 2]
        # 返回处理后的张量
        return x

# 创建一个随机数值为 [0, 1) 的形状为 (1, 2, 5) 的张量 X
X = torch.rand(1, 2, 5)
# 实例化一个 FourierLayer 的网络对象 net
net = FourierLayer()
# 将 X 输入到网络 net 中进行前向计算，并输出结果
print(X)
print(net(X))

tensor([[[0.2581, 0.3637, 0.3915, 0.2914, 0.4816],
         [0.5522, 0.5383, 0.2517, 0.0116, 0.4663]]])
tensor([[[ 3.6066+0.0000j,  0.6166-0.1563j],
         [-0.0338+0.0000j, -0.6828+0.2629j]]])
